# Data Wrangling - Order Data
This notebook processes order/sales data with marketing attribution.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
from datetime import datetime, timedelta
import re

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('pastel')

## Utility Functions

In [ ]:
def missing_values(df, percentage):
    """
    Drop columns with missing values above the specified percentage.
    """
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({
        'column_name': df.columns,
        'percent_missing': percent_missing
    })
    
    missing_drop = missing_value_df[missing_value_df.percent_missing > percentage]['column_name'].tolist()
    print(f"Dropping {len(missing_drop)} columns with >{percentage}% missing values")
    
    return df.drop(missing_drop, axis=1)

## Marketing Channel Mapping

In [ ]:
# Define paid media vendors
paidmedia_vendors_dict = {
    'Google': ['google', 'googleads'],
    'Netline': ['netline'],
    'Twitter': ['twitter'],
    'LinkedIn': ['linkedin'],
    'Facebook': ['facebook'],
    'eEffective': ['eeffective', 'retargeting'],
    'Industry Publication': ['gpsworld', 'xyht', 'precisionag'],
    'Event': ['xponential', 'intergeo', 'interdrone'],
    'Email Platform': ['mailchimp', 'constant contact']
}

# Define paid media mediums
paidmedia_medium_dict = {
    'SEM': ['cpc', 'sem', 'search'],
    'Retargeting': ['retargeting', 'display'],
    'Newsletter': ['enewsletter', 'newsletter'],
    'Email': ['eblast', 'email'],
    'Social': ['twitter', 'linkedin', 'instagram', 'facebook', 'social'],
    'Online Ad': ['banner', 'digital', 'onlinead'],
    'Print Ad': ['print', 'printad'],
    'Lead Generation': ['leadgeneration', 'whitepaper'],
    'Event': ['event', 'tradeshow'],
    'PR': ['pr', 'news'],
    'Webinar': ['webinar'],
    'Organic': ['referral', 'direct', 'organic']
}

print("Marketing channel mappings defined")
print(f"Vendors: {len(paidmedia_vendors_dict)}")
print(f"Mediums: {len(paidmedia_medium_dict)}")

## Generate Sample Order Data

In [ ]:
# Generate sample order data
np.random.seed(42)

# Define date range
start_date = datetime(2018, 1, 1)
end_date = datetime(2018, 12, 31)
date_range = pd.date_range(start_date, end_date, freq='D')

# Define channels
order_mediums = list(paidmedia_medium_dict.keys())
order_vendors = list(paidmedia_vendors_dict.keys())

# Create sample data
n_orders = 800
order_data = {
    'Order Date': np.random.choice(date_range, n_orders),
    'Order Number': [f'SO-{10000 + i}' for i in range(n_orders)],
    'Order Amount': np.random.uniform(500, 50000, n_orders).round(2),
    'Net Amount': np.random.uniform(400, 45000, n_orders).round(2),
    'Orders Medium': np.random.choice(order_mediums, n_orders),
    'Orders Vendor': np.random.choice(order_vendors, n_orders),
    'Discovery Source': np.random.choice(
        ['Web Search', 'Social Media', 'Email', 'Direct', 'Referral', 'Event'],
        n_orders
    ),
    'Product Category': np.random.choice(
        ['GNSS Receivers', 'Software Licenses', 'Services', 'Accessories'],
        n_orders
    ),
    'Customer Type': np.random.choice(
        ['New', 'Existing', 'Returning'],
        n_orders,
        p=[0.3, 0.5, 0.2]
    )
}

orders_df = pd.DataFrame(order_data)

# Add fulfillment time (workdays)
orders_df['Workdays To Fulfill'] = np.random.randint(1, 15, n_orders)

# Create order created date (slightly before order date)
orders_df['Order Created Date'] = orders_df['Order Date'] - pd.to_timedelta(
    np.random.randint(0, 5, n_orders), unit='D'
)

print(f"Generated {len(orders_df)} order records")
print(f"Date range: {orders_df['Order Date'].min()} to {orders_df['Order Date'].max()}")
orders_df.head(10)

## Data Cleaning and Validation

In [ ]:
# Check for missing values
print("Missing values per column:")
print(orders_df.isnull().sum())

# Remove invalid orders
orders_valid_df = orders_df[
    (orders_df['Order Amount'] > 0) & 
    (orders_df['Net Amount'] > 0)
].copy()

print(f"\nValid records: {len(orders_valid_df)} out of {len(orders_df)}")
print(f"Records removed: {len(orders_df) - len(orders_valid_df)}")

## Add Time-based Features

In [ ]:
# Create month-based features
orders_valid_df['Order Month'] = orders_valid_df['Order Date'].dt.to_period('M').dt.to_timestamp()
orders_valid_df['Year'] = orders_valid_df['Order Date'].dt.year
orders_valid_df['Quarter'] = orders_valid_df['Order Date'].dt.quarter
orders_valid_df['Month_Name'] = orders_valid_df['Order Date'].dt.strftime('%B')
orders_valid_df['Day_of_Week'] = orders_valid_df['Order Date'].dt.day_name()

print("Date features added successfully")
orders_valid_df[['Order Date', 'Order Month', 'Year', 'Quarter', 'Month_Name']].head()

## Aggregate by Medium

In [ ]:
# Group by order medium
orders_by_medium_df = orders_valid_df.groupby('Orders Medium').agg({
    'Order Amount': ['sum', 'mean', 'count'],
    'Net Amount': 'sum',
    'Workdays To Fulfill': 'mean'
}).round(2)

orders_by_medium_df.columns = ['Total Revenue', 'Avg Order Value', 'Order Count', 
                                'Total Net Amount', 'Avg Fulfillment Days']
orders_by_medium_df = orders_by_medium_df.sort_values('Total Revenue', ascending=False)
orders_by_medium_df = orders_by_medium_df.reset_index()

print("\nOrders Summary by Medium:")
print(orders_by_medium_df)

## Aggregate by Month

In [ ]:
# Group by month
orders_by_month_df = orders_valid_df.groupby('Order Month').agg({
    'Order Amount': ['sum', 'mean', 'count'],
    'Net Amount': 'sum'
}).round(2)

orders_by_month_df.columns = ['Total Revenue', 'Avg Order Value', 'Order Count', 'Total Net Amount']
orders_by_month_df = orders_by_month_df.reset_index()

print("\nOrders Summary by Month:")
print(orders_by_month_df)

## Visualizations

In [ ]:
# Revenue by Marketing Medium
fig, ax = plt.subplots(figsize=(12, 6))

sns.barplot(
    data=orders_by_medium_df,
    x='Orders Medium',
    y='Total Revenue',
    ax=ax,
    palette='viridis'
)

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax.set_title('Revenue by Marketing Medium', fontsize=16, fontweight='bold')
ax.set_xlabel('Marketing Medium', fontsize=12)
ax.set_ylabel('Total Revenue ($)', fontsize=12)

plt.tight_layout()
plt.show()

print(f"\nTotal Revenue: ${orders_by_medium_df['Total Revenue'].sum():,.2f}")

In [ ]:
# Monthly Revenue Trend
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(
    orders_by_month_df['Order Month'],
    orders_by_month_df['Total Revenue'],
    marker='o',
    linewidth=2.5,
    markersize=8,
    color='darkgreen',
    label='Revenue'
)

ax.fill_between(
    orders_by_month_df['Order Month'],
    orders_by_month_df['Total Revenue'],
    alpha=0.3,
    color='darkgreen'
)

# Add trend line
z = np.polyfit(range(len(orders_by_month_df)), orders_by_month_df['Total Revenue'], 1)
p = np.poly1d(z)
ax.plot(
    orders_by_month_df['Order Month'],
    p(range(len(orders_by_month_df))),
    "r--",
    alpha=0.8,
    linewidth=2,
    label='Trend'
)

ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax.set_title('Monthly Revenue Trend', fontsize=16, fontweight='bold')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Revenue ($)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Order Count by Medium
fig, ax = plt.subplots(figsize=(10, 6))

wedges, texts, autotexts = ax.pie(
    orders_by_medium_df['Order Count'],
    labels=orders_by_medium_df['Orders Medium'],
    autopct='%1.1f%%',
    startangle=90,
    colors=sns.color_palette('Set3', len(orders_by_medium_df))
)

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(10)

ax.set_title('Order Distribution by Marketing Medium', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Customer Type Analysis
customer_analysis = orders_valid_df.groupby(['Customer Type', 'Orders Medium']).agg({
    'Order Amount': 'sum'
}).reset_index()

fig, ax = plt.subplots(figsize=(12, 6))

pivot_data = customer_analysis.pivot(index='Orders Medium', 
                                      columns='Customer Type', 
                                      values='Order Amount')
pivot_data.plot(kind='bar', stacked=True, ax=ax, 
                color=['#FF6B6B', '#4ECDC4', '#45B7D1'])

ax.set_title('Revenue by Customer Type and Medium', fontsize=16, fontweight='bold')
ax.set_xlabel('Marketing Medium', fontsize=12)
ax.set_ylabel('Revenue ($)', fontsize=12)
ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax.legend(title='Customer Type', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Performance Metrics Dashboard
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Average Order Value by Medium
aov_sorted = orders_by_medium_df.sort_values('Avg Order Value', ascending=True)
axes[0, 0].barh(aov_sorted['Orders Medium'], aov_sorted['Avg Order Value'], color='skyblue')
axes[0, 0].set_title('Avg Order Value by Medium', fontweight='bold')
axes[0, 0].set_xlabel('AOV ($)')
axes[0, 0].xaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))

# Fulfillment Time by Medium
fulfill_sorted = orders_by_medium_df.sort_values('Avg Fulfillment Days', ascending=True)
axes[0, 1].barh(fulfill_sorted['Orders Medium'], fulfill_sorted['Avg Fulfillment Days'], 
                color='lightcoral')
axes[0, 1].set_title('Avg Fulfillment Time by Medium', fontweight='bold')
axes[0, 1].set_xlabel('Days')

# Revenue by Product Category
product_revenue = orders_valid_df.groupby('Product Category')['Order Amount'].sum().sort_values()
axes[1, 0].barh(product_revenue.index, product_revenue.values, color='lightgreen')
axes[1, 0].set_title('Revenue by Product Category', fontweight='bold')
axes[1, 0].set_xlabel('Revenue ($)')
axes[1, 0].xaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))

# Orders by Day of Week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_counts = orders_valid_df.groupby('Day_of_Week')['Order Number'].count().reindex(day_order)
axes[1, 1].bar(range(len(dow_counts)), dow_counts.values, color='plum')
axes[1, 1].set_xticks(range(len(dow_counts)))
axes[1, 1].set_xticklabels([d[:3] for d in dow_counts.index], rotation=0)
axes[1, 1].set_title('Orders by Day of Week', fontweight='bold')
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('Order Count')

plt.tight_layout()
plt.show()

## Save Processed Data

In [ ]:
# Save to CSV
orders_valid_df.to_csv('orders_data_processed.csv', index=False)
orders_by_medium_df.to_csv('orders_by_medium.csv', index=False)
orders_by_month_df.to_csv('orders_by_month.csv', index=False)

print("Data saved successfully!")
print(f"- orders_data_processed.csv ({len(orders_valid_df)} records)")
print(f"- orders_by_medium.csv ({len(orders_by_medium_df)} records)")
print(f"- orders_by_month.csv ({len(orders_by_month_df)} records)")